In [9]:
import argparse
from itertools import cycle
import torch
import typing
import pandas as pd
from torch.utils.data import DataLoader, Dataset, IterableDataset, random_split
import numpy as np
from encoder import Encoder, create_encoder
from typing import Generator
from model import GPTTransformer
from utils import Config
from data import prepare_data

# Load Model

In [10]:
config = Config(
    epoch=10,
    learning_rate=1e-3,
    batch_size=128,
    weight_decay=1e-5,
    seq_len=128,
    d_embed=64,
    n_layers=6,
    n_heads=8,
    dropout=0.2,
)

_, _, vocab_size = prepare_data(
    batch_size=config.batch_size,
    seq_len=config.seq_len,
)

encoder = create_encoder("./data/pg16457.txt", 1000)

config.vocab_size = vocab_size

model = GPTTransformer.load_from_checkpoint('./lightning_logs/version_0/checkpoints/epoch=9-step=550.ckpt', config=config)

In [90]:
@torch.no_grad()
def greedy_decode(model, max_tokens, device):
  idx = torch.tensor([[0]]).long().to(device)

  model.eval()

  for _ in range(max_tokens):
    src_mask = torch.triu(torch.ones(idx.shape[-1], idx.shape[-1]) * float('-inf'), diagonal=1).to(device)
    logits = model(idx, src_mask, None) # no padding token in inference mode.
    next_token_logits = logits[:, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    next_tokens = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, next_tokens), dim=1)

    if idx.squeeze()[-1] == 1 or idx.squeeze()[-1] == 2:
      break
  
  if (idx.squeeze().numel() < 1):
    return

  idx = idx.squeeze()[1:] # trim away the <bos> token.
  idx = idx.squeeze()[:-1] # trim away the <eos> token.
  
  print(''.join([itos[i] for i in idx.tolist()]))

torch.Size([128, 512])
